# This demonstrates the following:
How to load imdb data
How load converts words into integer indexes
How words are stored in highest occuring freq to lowest occurring
How to take top 5000 or 10000 vocbualry size
idiosyncracy of imdb.get_word_index() 
Re-convert the index to take into account padding, start, out-of-vocabulary
Create a CNN wtih multiple kernel size (corresponding to 1-gram, 2-gram, 3-gram)
Use functional Keras

In [1]:
import tensorflow as tf
print (tf.__version__)

c:\python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.11.0


In [2]:
import keras
from keras.datasets import imdb
from keras.layers import Dense,  Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling1D,MaxPooling2D

Using TensorFlow backend.


In [3]:
print (keras.__version__)
vocabulary_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('IMDB dataset  {} training samples, {} test samples'.format(len(X_train), len(X_test)))

2.2.4
IMDB dataset  25000 training samples, 25000 test samples


In [4]:
print('---Features: review in number sequence After padding---')
print(X_train[6])
print('Sentiment 1 = positve, 0 = negative')
print(y_train[6])

---Features: review in number sequence After padding---
[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
Sentiment 1 = positve, 0 = negative
1


In [5]:
w2id = imdb.get_word_index() 
id2word = {i: word  for word, i in w2id.items()}
print (len(w2id), id2word[1], w2id["the"])

88584 the 1


In [6]:
#Jibberish data?
actual_data = []
for i in range (len(X_train[6])):
    ind = X_train[6][i]
    if(ind != 0):
        actual_data.append(id2word[ind])
actual_data = " ".join (actual_data )
print (actual_data, "\nAnd sentiment is ", y_train[6])

the boiled full involving to impressive boring this as murdering naschy br villain and suggestion need has of costumes b message to may of props this and concentrates concept issue skeptical to god's he is and unfolds movie women like isn't surely i'm and to toward in here's for from did having because very quality it is and starship really book is both too worked carl of and br of reviewer closer figure really there will originals things is far this make mistakes and was couldn't of few br of you to don't female than place she to was between that nothing dose movies get are and br yes female just its because many br of overly to descent people time very bland 
And sentiment is  1


In [7]:
word2id ={w: i+3 for w, i in w2id.items()}
word2id["__PADDING__"] = 0
word2id["__START__"] = 1
word2id["__UNK__"] = 2

#This returns the index of the words from 1 to n with 1 being the most frequently occuring word, 

# and n the least frequently occuring word

print (type(word2id), len (word2id))

id2word = {i: word  for word, i in word2id.items()}
print(id2word[1], id2word[2], #id2word[3],#
      id2word[4], id2word[5], id2word[6], id2word[88586])
actual_data = []
for i in range (len(X_train[6])):
    ind = X_train[6][i]
    if(ind != 0):
        actual_data.append(id2word[ind])
actual_data = " ".join (actual_data )
print (actual_data, "\nAnd sentiment is ", y_train[6])
print (word2id["the"])

<class 'dict'> 88587
__START__ __UNK__ the and a voorhees'
__START__ lavish production values and solid performances in this straightforward adaption of jane __UNK__ satirical classic about the marriage game within and between the classes in __UNK__ 18th century england northam and paltrow are a __UNK__ mixture as friends who must pass through __UNK__ and lies to discover that they love each other good humor is a __UNK__ virtue which goes a long way towards explaining the __UNK__ of the aged source material which has been toned down a bit in its harsh __UNK__ i liked the look of the film and how shots were set up and i thought it didn't rely too much on __UNK__ of head shots like most other films of the 80s and 90s do very good results 
And sentiment is  1
4


In [8]:
print('Maximum review length: {}'.format(
len((max((X_train + X_test), key=len)))))

Maximum review length: 2697


In [9]:
from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
modelCNN = None
modelRNN=None
modelLSTM=None

In [37]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,  Concatenate, MaxPool2D, MaxPool1D, Reshape,GlobalMaxPooling1D
from keras.models import Sequential,Model
from keras import Input
from keras.layers.advanced_activations import ReLU
from keras.layers import Activation
from keras import regularizers
import numpy as np

In [11]:
def CreateModel2D3Nets():
    #embedding_size=32

    embedding_size=50
    deep_inputs = Input(shape=(max_words,))
    print ("deep_inputs.shape", deep_inputs.shape)
    x = Embedding(vocabulary_size, embedding_size, input_length=max_words)(deep_inputs)
    x = Reshape((max_words,embedding_size,1) )(x)
    print ("x.shape", x.shape)
#     x = np.reshape(max_words, embedding_size, 1)(x)
#     print (x.shape)
    x1  = Conv2D(filters=100, kernel_size=(5, embedding_size),  activation="relu")(x)
    print ("x1.shape", x1.shape)
    x1Pool = MaxPool2D (pool_size=(max_words-4, 1))(x1)    
    print ("x1Pool.shape", x1Pool.shape) 
    x2 =  Conv2D(filters=100, kernel_size=(2, embedding_size),  activation="relu")(x)
    print ("x2.shape", x2.shape)
    x2Pool = MaxPool2D (pool_size=(max_words-1, 1))(x2)    
    print ("x2Pool.shape", x2Pool.shape)    
    x3 =  Conv2D(filters=100, kernel_size=(3, embedding_size),  activation="relu")(x)
    print ("x3.shape", x3.shape)
    x3Pool = MaxPool2D (pool_size=(max_words-2, 1))(x3)    
    print ("x3Pool.shape", x3Pool.shape)    
    xAll = Concatenate(axis = 1)([x1Pool, x2Pool, x3Pool])
    print ("xAll.shape", xAll.shape)
#     xAll = Conv2D(filters=16, )
    
    xAll = Flatten()(xAll)
    print ("xAll.shape", xAll)
    output = Dense(1, activation= "sigmoid")(xAll)
    print ("output.shape", output.shape)
    modelCNN = Model(inputs = deep_inputs, outputs = output) 
    modelCNN.summary()
    return modelCNN

In [12]:
def CreateModel2D2Nets():
    #embedding_size=32

    embedding_size=50
    deep_inputs = Input(shape=(max_words,))
    print ("deep_inputs.shape", deep_inputs.shape)
    x = Embedding(vocabulary_size, embedding_size, input_length=max_words)(deep_inputs)
    x = Reshape((max_words,embedding_size,1) )(x)
    print ("x.shape", x.shape)
#     x = np.reshape(max_words, embedding_size, 1)(x)
#     print (x.shape)
    x1  = Conv2D(filters=100, kernel_size=(5, embedding_size),  activation="relu")(x)
    print ("x1.shape", x1.shape)
    x1Pool = MaxPool2D (pool_size=(max_words-4, 1))(x1)    
    print ("x1Pool.shape", x1Pool.shape) 
    x2 =  Conv2D(filters=100, kernel_size=(2, embedding_size),  activation="relu")(x)
    print ("x2.shape", x2.shape)
    x2Pool = MaxPool2D (pool_size=(max_words-1, 1))(x2)    
    print ("x2Pool.shape", x2Pool.shape)    
    xAll = Concatenate(axis = 1)([x1Pool, x2Pool])
    print ("xAll.shape", xAll.shape)
#     xAll = Conv2D(filters=16, )
    
    xAll = Flatten()(xAll)
    print ("xAll.shape", xAll)
    output = Dense(1, activation= "sigmoid")(xAll)
    print ("output.shape", output.shape)
    modelCNN = Model(inputs = deep_inputs, outputs = output) 
    modelCNN.summary()
    return modelCNN

In [13]:
model2NetsCNN = CreateModel2D2Nets()

deep_inputs.shape (?, 500)
x.shape (?, 500, 50, 1)
x1.shape (?, 496, 1, 100)
x1Pool.shape (?, 1, 1, 100)
x2.shape (?, 499, 1, 100)
x2Pool.shape (?, 1, 1, 100)
xAll.shape (?, 2, 1, 100)
xAll.shape Tensor("flatten_1/Reshape:0", shape=(?, ?), dtype=float32)
output.shape (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 500, 50)      500000      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 500, 50, 1)   0           embedding_1[0][0]                
________________________________

In [14]:
model3NetsCNN = CreateModel2D3Nets()

deep_inputs.shape (?, 500)
x.shape (?, 500, 50, 1)
x1.shape (?, 496, 1, 100)
x1Pool.shape (?, 1, 1, 100)
x2.shape (?, 499, 1, 100)
x2Pool.shape (?, 1, 1, 100)
x3.shape (?, 498, 1, 100)
x3Pool.shape (?, 1, 1, 100)
xAll.shape (?, 3, 1, 100)
xAll.shape Tensor("flatten_2/Reshape:0", shape=(?, ?), dtype=float32)
output.shape (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 50)      500000      input_2[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 500, 50, 1)   0           embedding_2[

In [15]:
def CreateModel1D():
    #embedding_size=32

    embedding_size=50
    deep_inputs = Input(shape=(max_words,))
    print ("deep_inputs.shape", deep_inputs.shape)
    x = Embedding(vocabulary_size, embedding_size, input_length=max_words)(deep_inputs)
    x = Reshape((max_words,embedding_size) )(x)
    print ("x.shape", x.shape)
#     x = np.reshape(max_words, embedding_size, 1)(x)
#     print (x.shape)
    x1  = Conv1D(filters=100, kernel_size=5,  activation="relu")(x)
    print ("x1.shape", x1.shape)
    x1Pool = MaxPool1D (pool_size=5)(x1)    
    print ("x1Pool.shape", x1Pool.shape) 
    x2 =  Conv1D(filters=100, kernel_size=2,  activation="relu")(x)
    print ("x2.shape", x2.shape)
    x2Pool = MaxPool1D (pool_size=2)(x2)    
    print ("x2Pool.shape", x2Pool.shape)    
    x3 =  Conv1D(filters=100, kernel_size=3,  activation="relu")(x)
    print ("x3.shape", x3.shape)
    x3Pool = MaxPool1D (pool_size=3)(x3)    
    print ("x3Pool.shape", x3Pool.shape)    
    xAll = Concatenate(axis = 1)([x1Pool, x2Pool, x3Pool])
    print ("xAll.shape", xAll.shape)
#     
    xAll = Conv1D(filters=150, kernel_size = 5, activation="relu" ) (xAll)
    xAll = MaxPool1D(pool_size=5) (xAll)

    xAll = Conv1D(filters=200, kernel_size = 5, activation="relu" ) (xAll)
    xAll = MaxPool1D(pool_size=5) (xAll)
    
    xAll = Flatten()(xAll)
    print ("xAll.shape", xAll)
    output = Dense(1, activation= "sigmoid")(xAll)
    print ("output.shape", output.shape)
    modelCNN = Model(inputs = deep_inputs, outputs = output) 
    modelCNN.summary()
    return modelCNN
model1DCNN = CreateModel1D()


deep_inputs.shape (?, 500)
x.shape (?, 500, 50)
x1.shape (?, 496, 100)
x1Pool.shape (?, 99, 100)
x2.shape (?, 499, 100)
x2Pool.shape (?, 249, 100)
x3.shape (?, 498, 100)
x3Pool.shape (?, 166, 100)
xAll.shape (?, 514, 100)
xAll.shape Tensor("flatten_3/Reshape:0", shape=(?, ?), dtype=float32)
output.shape (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 500, 50)      500000      input_3[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 500, 50)      0           embedding_3[0][0]            

In [16]:
model3NetsCNN.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [17]:
model2NetsCNN.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [18]:
batch_size = 256
num_epochs = 10
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

In [19]:
callback_list = [#early1
                keras.callbacks.ModelCheckpoint(filepath="my_CNNmod.h5", monitor="val_acc",
                                               save_best_only=True),
                #keras.callbacks.TerminateOnNaN()
                ]

In [20]:
model3NetsCNN.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, 
             epochs=num_epochs, callbacks=callback_list)

Train on 24744 samples, validate on 256 samples
Epoch 1/10
24744/24744 [==============================] - 142s 6ms/step - loss: 0.5803 - acc: 0.7072 - val_loss: 0.2982 - val_acc: 0.8789
Epoch 2/10
24744/24744 [==============================] - 141s 6ms/step - loss: 0.3008 - acc: 0.8763 - val_loss: 0.2298 - val_acc: 0.8984
Epoch 3/10
24744/24744 [==============================] - 148s 6ms/step - loss: 0.2012 - acc: 0.9252 - val_loss: 0.2254 - val_acc: 0.8906
Epoch 4/10
24744/24744 [==============================] - 159s 6ms/step - loss: 0.1406 - acc: 0.9522 - val_loss: 0.2209 - val_acc: 0.8984
Epoch 5/10
24744/24744 [==============================] - 204s 8ms/step - loss: 0.0946 - acc: 0.9719 - val_loss: 0.2459 - val_acc: 0.8984
Epoch 6/10
24744/24744 [==============================] - 208s 8ms/step - loss: 0.0602 - acc: 0.9851 - val_loss: 0.2703 - val_acc: 0.8906
Epoch 7/10
24744/24744 [==============================] - 207s 8ms/step - loss: 0.0377 - acc: 0.9931 - val_loss: 0.2863 - va

In [21]:
model2NetsCNN.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, 
             epochs=num_epochs, callbacks=callback_list)

Train on 24744 samples, validate on 256 samples
Epoch 1/10
24744/24744 [==============================] - 150s 6ms/step - loss: 0.5945 - acc: 0.6908 - val_loss: 0.3346 - val_acc: 0.8594
Epoch 2/10
24744/24744 [==============================] - 142s 6ms/step - loss: 0.3180 - acc: 0.8678 - val_loss: 0.2215 - val_acc: 0.9023
Epoch 3/10
24744/24744 [==============================] - 175s 7ms/step - loss: 0.2133 - acc: 0.9176 - val_loss: 0.2054 - val_acc: 0.9141
Epoch 4/10
24744/24744 [==============================] - 157s 6ms/step - loss: 0.1482 - acc: 0.9486 - val_loss: 0.2110 - val_acc: 0.9141
Epoch 5/10
24744/24744 [==============================] - 148s 6ms/step - loss: 0.1008 - acc: 0.9693 - val_loss: 0.2207 - val_acc: 0.9023
Epoch 6/10
24744/24744 [==============================] - 143s 6ms/step - loss: 0.0661 - acc: 0.9835 - val_loss: 0.2333 - val_acc: 0.9102
Epoch 7/10
24744/24744 [==============================] - 135s 5ms/step - loss: 0.0412 - acc: 0.9925 - val_loss: 0.2484 - va

In [29]:
model1DCNN.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
model1DCNN.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, 
             epochs=num_epochs, callbacks=callback_list)
bestCNN1D = CreateModel()
bestCNN1D.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
bestCNN1D.load_weights("my_CNNmod.h5", by_name=False)
scoresFinal1D = modelCNN.evaluate(X_test, y_test, verbose=0)
scoresBest1D = bestCNN.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scoresFinal1D, scoresBest1D)

Train on 24744 samples, validate on 256 samples
Epoch 1/10
24744/24744 [==============================] - 330s 13ms/step - loss: 0.5205 - acc: 0.6949 - val_loss: 0.1966 - val_acc: 0.9258
Epoch 2/10
24744/24744 [==============================] - 358s 14ms/step - loss: 0.2073 - acc: 0.9198 - val_loss: 0.1640 - val_acc: 0.9258
Epoch 3/10
24744/24744 [==============================] - 339s 14ms/step - loss: 0.1282 - acc: 0.9535 - val_loss: 0.2043 - val_acc: 0.9102
Epoch 4/10
24744/24744 [==============================] - 319s 13ms/step - loss: 0.0696 - acc: 0.9764 - val_loss: 0.2330 - val_acc: 0.8945
Epoch 5/10
24744/24744 [==============================] - 319s 13ms/step - loss: 0.0368 - acc: 0.9882 - val_loss: 0.3412 - val_acc: 0.8984
Epoch 6/10
24744/24744 [==============================] - 322s 13ms/step - loss: 0.0156 - acc: 0.9952 - val_loss: 0.5031 - val_acc: 0.8906
Epoch 7/10
24744/24744 [==============================] - 320s 13ms/step - loss: 0.0037 - acc: 0.9992 - val_loss: 0.63

In [16]:
bestCNN = CreateModel()
bestCNN.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
bestCNN.load_weights("my_CNNmod.h5", by_name=False)
scoresFinal = modelCNN.evaluate(X_test, y_test, verbose=0)
scoresBest = bestCNN.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scoresFinal, scoresBest)

deep_inputs.shape (?, 500)
x.shape (?, 500, 50, 1)
x1.shape (?, 496, 1, 100)
x1Pool.shape (?, 1, 1, 100)
x2.shape (?, 499, 1, 100)
x2Pool.shape (?, 1, 1, 100)
x3.shape (?, 498, 1, 100)
x3Pool.shape (?, 1, 1, 100)
xAll.shape (?, 3, 1, 100)
xAll.shape Tensor("flatten_2/Reshape:0", shape=(?, ?), dtype=float32)
output.shape (?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 50)      500000      input_2[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 500, 50, 1)   0           embedding_2[

In [22]:
reviewList = ["the movie was boring", "the movie was not too boring",
              "the movie was not too long",
              "the movie was not at all bad", 
              "the movie was a total waste of my time",
              "the food was so deliciously delicious that i felt sinfully wicked" ,   
              "the food was so wicked that after eating i felt sinful"
             ]
print ("Now going to predict these reviews")

Now going to predict these reviews


In [27]:
reviewList1 = ["the movie was boring", "I cannot watch such a movie",
              "the food was so wicked that after eating i felt sinful",
              "the meal was tasty",
              "the dessert was suptuous"
             ]
print ("Now going to predict these reviews")

Now going to predict these reviews


In [23]:
print (word2id["the"])

4


In [25]:
def PredictSentiment(reviewList, ModelListTuple):
    sentiment= {True: "Positive",
               False: "Negative"}
    Threshold = 0.5
    for r in reviewList:
        words = r.split()
        review = []
        print ("review=", r)
        for word in words:
          if word not in word2id: 
            review.append(2)
            print (word, "Appended 2")
          else:
            if (word2id[word]) >= vocabulary_size:
                print("got a word outside teh vocab_index", word, word2id[word], "breaking")
                break
            print ("word=", word, "id=", word2id[word])
            review.append(word2id[word]) 
        review = keras.preprocessing.sequence.pad_sequences([review],
          truncating='pre', padding='pre', maxlen=max_words)
        for i,m in enumerate(ModelListTuple):
            if m[0] is not None:
                prediction = m[0].predict(review)
                print("Prediction Probability for ", "\"",r, "\"",ModelListTuple[i][1]," = ", prediction[0][0], "Sentiment=", 
                      sentiment[prediction[0][0]>Threshold], "\n")
                
PredictSentiment(reviewList,[(model3NetsCNN, "3 Concat CNN"), (model2NetsCNN, "2 Concat Nets CNN")])

review= the movie was boring
word= the id= 4
word= movie id= 20
word= was id= 16
word= boring id= 357
Prediction Probability for  " the movie was boring " 3 Concat CNN  =  0.005292458 Sentiment= Negative 

Prediction Probability for  " the movie was boring " 2 Concat Nets CNN  =  0.0020353186 Sentiment= Negative 

review= the movie was not too boring
word= the id= 4
word= movie id= 20
word= was id= 16
word= not id= 24
word= too id= 99
word= boring id= 357
Prediction Probability for  " the movie was not too boring " 3 Concat CNN  =  0.00031824218 Sentiment= Negative 

Prediction Probability for  " the movie was not too boring " 2 Concat Nets CNN  =  4.344389e-05 Sentiment= Negative 

review= the movie was not too long
word= the id= 4
word= movie id= 20
word= was id= 16
word= not id= 24
word= too id= 99
word= long id= 196
Prediction Probability for  " the movie was not too long " 3 Concat CNN  =  0.0006158856 Sentiment= Negative 

Prediction Probability for  " the movie was not too long 

In [28]:
PredictSentiment(reviewList1,[(model3NetsCNN, "3 Concat CNN"), (model2NetsCNN, "2 Concat Nets CNN")])

review= the movie was boring
word= the id= 4
word= movie id= 20
word= was id= 16
word= boring id= 357
Prediction Probability for  " the movie was boring " 3 Concat CNN  =  0.005292458 Sentiment= Negative 

Prediction Probability for  " the movie was boring " 2 Concat Nets CNN  =  0.0020353186 Sentiment= Negative 

review= I cannot watch such a movie
I Appended 2
word= cannot id= 566
word= watch id= 106
word= such id= 141
word= a id= 6
word= movie id= 20
Prediction Probability for  " I cannot watch such a movie " 3 Concat CNN  =  0.2745939 Sentiment= Negative 

Prediction Probability for  " I cannot watch such a movie " 2 Concat Nets CNN  =  0.06950959 Sentiment= Negative 

review= the food was so wicked that after eating i felt sinful
word= the id= 4
word= food id= 1644
word= was id= 16
word= so id= 38
word= wicked id= 3799
word= that id= 15
word= after id= 103
word= eating id= 1889
word= i id= 13
word= felt id= 421
got a word outside teh vocab_index sinful 17010 breaking
Prediction Pr

In [88]:
def cos_similarity(vec_u, vec_v):
    import numpy as np
    distance = 0.0   
    dot = np.inner(vec_u,vec_v)
    norm_vec_u = np.linalg.norm(vec_u)
    norm_vec_v = np.linalg.norm(vec_v)
    cos_similarity_val = dot/(norm_vec_u*norm_vec_v)
    
    return cos_similarity_val

In [63]:
from keras.models import Model
model = modelCNN
layer_outputs = [layer.output for layer in model.layers]
i = 0
print ("#Layers = ", len(layer_outputs))
print (type(model.layers[0]))
l = model.layers[0]
print (len(l.get_weights()))
print (l.get_weights()[0].shape)
print (word2id["apple"], word2id["orange"], "word of 6335", id2word[6335], "word of 6922", id2word[6922])
print ("apple", l.get_weights()[0][word2id["apple"]])
print ("orange", l.get_weights()[0][word2id["orange"]])
print ("delicious", l.get_weights()[0][6335])
print ("deliciously", l.get_weights()[0][6922])
print ("Similarity between delicious and deliciously", cos_similarity(l.get_weights()[0][6335], l.get_weights()[0][6922]))
print ("Similarity between apple and orange", cos_similarity(l.get_weights()[0][word2id["apple"]], 
                                                             l.get_weights()[0][word2id["orange"]]))
print ("Similarity between apple and delicious", cos_similarity(l.get_weights()[0][word2id["apple"]], 
                                                             l.get_weights()[0][6335]))
print ("Similarity between tasty and delicious", cos_similarity(l.get_weights()[0][word2id["tasty"]], 
                                                             l.get_weights()[0][word2id["delicious"]]))


#Layers =  17
<class 'keras.engine.input_layer.InputLayer'>
0


IndexError: list index out of range

In [ ]:
#Now use patience, and early stopping
#early1 = keras.callbacks.EarlyStopping(monitor ="val_acc", patience=1, restore_best_weights=True)
#early1 = keras.callbacks.EarlyStopping(monitor ="acc", patience=2, baseline = 0.95)
#early1 = keras.callbacks.EarlyStopping(monitor ="val_acc", patience=1, restore_best_weights=True)

callback_list = [#early1
                keras.callbacks.ModelCheckpoint(filepath="my_mod2.h5", monitor="val_acc",
                                               save_best_only=True),
                #keras.callbacks.TerminateOnNaN()
                ]

In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
X_train3 = X_train2[:1000]
Y_train3 = y_train2[:1000]
print (X_train3.shape, Y_train3.shape)

In [ ]:
model.fit(X_train3, Y_train3, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=15, #num_epochs,
             callbacks=callback_list)

In [ ]:
PredictSentiment(reviewList,[(modelRNN, "RNN"), (model, "CNN Checkpoint"), (modelLSTM, "LSTM")])